# Lab 4.2 : Spark SQL

### Overview
Using SQL statements with Spark SQL.   

### Depends On 
None

### Run time
20-30 mins


In [ ]:
try:
    spark
except NameError:
    # initialize Spark Session
    import os
    import sys
    top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
    if top_dir not in sys.path:
        sys.path.append(top_dir)

    from init_spark import init_spark
    spark = init_spark()

print('Spark UI running on port ' + spark.sparkContext.uiWebUrl.split(':')[2])
spark

## Spark application UI (4040+)


**As you run through the lab keep an eye on Spark application ui on port 4040+.  Inspect how each operation is executed!**

![](../assets/images/5.2c.png)

## STEP 1: Load Clickstream data

In [ ]:
clickstreamDF = spark.read.json("../data/click-stream/clickstream.json")
## FOR CLOUD ACCOUNTS USE THIS
# clickstreamDF = spark.read.json("s3://elephantscale-public/data/click-stream/clickstream.json")

print(clickstreamDF)
clickstreamDF.show()

## STEP 2: Querying using SQL

**==> Register dataframe as a table**

In [ ]:
clickstreamDF.createOrReplaceTempView("clickstream")
print("created clickstream temp table")

In [ ]:
logs = spark.sql("select * from clickstream")
#'logs' is a dataframe

# display the logs, hint : show
logs.show()

# or one liner
# spark.sql("select * from clickstream").show()

**==> Find records with only clicks**


In [ ]:
# Find Records with only clicks
# TODO: fix the  table name?
spark.sql("select * from ???table_name??? where action = '???'").show()


**==> Count records from each domain, sort the output by most to least**

Sample output:

```
+-----------------+-----+
|           domain|total|
+-----------------+-----+
|      nytimes.com|    1|
|     facebook.com|    1|
|       google.com|    2|
|       amazon.com|    2|
|    wikipedia.org|    3|
+-----------------+-----+
```

In [ ]:
# TODO: Try this query here:

spark.sql("select domain, COUNT(*) as total from clickstream group by ???  order by ??? ").show()

**==>Action: Inspect 4040+ UI.  And see how the execution happens**


## STEP 3: Joining Datasets

**==> Load `domains` dataset and register it to table `domains`**

In [ ]:
domainsDF = spark.read.json("../data/click-stream/domain-info.json")

## For cloud accounts use this
# domainsDF = spark.read.json("s3://elephantscale-public/data/click-stream/domain-info.json")

domainsDF.show()
domainsDF.createOrReplaceTempView("domains")

**==> Join `clickstreams` and `domains`**    
Hint : Join query syntax for joining two tables A, B on A.x and A.y is
```
spark.sql("select A.*, B.* from A  join B  ON (A.x = B.y)") 
```

In [ ]:
# Write sql query for joining clickstreams and domains
# TODO : fill in the common attribute for  clickstream & domain table
# Hint : domain

s = """
SELECT clickstream.*, domains.*  
from clickstream join domains 
ON (clickstream.???= domains.???)
"""

spark.sql(s).show()


**==> Count traffic per domain category (news, social ..etc)**    
Hint : query the joined datasets


In [ ]:
# TODO: Count traffic per domained category

s = """
SELECT  domains.category, count(*) as total  
from clickstream join domains 
ON (clickstream.domain= domains.domain)
group by ???
order by ???
"""

spark.sql(s).show()



## STEP 4: Explore UI
(Your DAG visualization may be slightly different from what we show here)

<img src="../assets/images/5.2c.png" style="border: 5px solid grey; max-width:100%;"/>

Look at SQL execution details

<img src="../assets/images/5.2d.png" style="border: 5px solid grey; max-width:100%;"/>


## Step 5 : Understand Query Execution
We will use **explain** 

In [ ]:
spark.sql("select * from clickstream where cost > 100").explain(extended=True)

In [ ]:
spark.sql("join query here").explain(extended=True)

### Notice the optimizations!

There are few optimizations here

* broadcast join
* filtering on `domain != null`
* pushed down predicates  `domain != null`

![](../assets/images/dataframe-7-explain.png)

## Step 6 - Bonus Lab - Adaptive Query Engine (AQE)

AQE is a new shiny SQL engine for Spark 3.  We will run some experiments with AQE.

Here is a [sample notebook](4.2b-aqe.ipynb) you can get started with.
